LCEL (LangChain Expression Language) is a domain-specific language in LangChain designed to simplify how developers create and manipulate prompts, chains, and outputs. It allows combining LLM calls, variables, and logic in a concise, readable syntax without writing full Python code. LCEL is useful for building complex workflows quickly, like conditional flows or multi-step reasoning, while keeping everything declarative. It improves maintainability by separating the chain logic from the code itself. Essentially, LCEL makes LangChain projects more modular, flexible, and easier to reason about.

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

REDACTED' | Set-Content replacements.txt'

In [7]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=groq_api_key) 
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F633C12CC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F633D56000>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage , SystemMessage
from sympy import content
messages = [
    SystemMessage(content="translate from english to french"),
    HumanMessage(content="hello how are you")
]

result = llm.invoke(messages)

In [10]:
result

AIMessage(content='Bonjour, comment vas-tu ? \n\n(Note: A more formal translation would be "Bonjour, comment allez-vous ?" but I used the informal "tu" since you used "you" in your greeting)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 44, 'total_tokens': 88, 'completion_time': 0.186088342, 'prompt_time': 0.018046498, 'queue_time': 0.087308932, 'total_time': 0.20413484}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9b36eef0-8f83-4715-b330-37c229949230-0', usage_metadata={'input_tokens': 44, 'output_tokens': 44, 'total_tokens': 88})

In [12]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment vas-tu ? \n\n(Note: A more formal translation would be "Bonjour, comment allez-vous ?" but I used the informal "tu" since you used "you" in your greeting)'

In [13]:
# using lcel - chain the components
chain = llm|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ?'

In [ ]:
#prompt template - convert into list of message 
from langchain_core.prompts import ChatPromptTemplate

template = "translate the following into {language}"

prompt = ChatPromptTemplate.from_messages(
    [("system",template),("user","{text}")]
)

In [19]:
result = prompt.invoke({"language":"french","text":"hello"})
result

ChatPromptValue(messages=[SystemMessage(content='translate the following into french', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello', additional_kwargs={}, response_metadata={})])

In [20]:
result.to_messages()

[SystemMessage(content='translate the following into french', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]

In [21]:
chain=prompt|llm|parser
chain.invoke({"language":"french","text":"hello"})

'Bonjour !'